In [1]:
import pandas as pd

import numpy as np


import pickle

In [2]:
games_df = pd.read_csv("game_data.csv")

In [4]:
games_df.head()

,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31,12,2012,600,8,Rapid,French Defense: Normal Variation,C00,13
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31,12,2012,480,2,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31,12,2012,420,17,Rapid,Four Knights Game: Italian Variation,C50,11
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31,12,2012,60,1,Bullet,Caro-Kann Defense: Goldman Variation,B12,47
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31,12,2012,60,1,Bullet,French Defense: La Bourdonnais Variation,C00,23


In [5]:
games_df.shape

(14918230, 16)

In [13]:
games_df.TimeControl.value_counts()

Blitz        6007902
Rapid        4358458
Bullet       4242183
Classical     309687
Name: TimeControl, dtype: int64

In [19]:
games_df[games_df.TimeControl=="Rapid"].WhiteName.unique().shape

(115886,)

In [20]:
games_df[games_df.TimeControl=="Rapid"].BlackName.unique().shape

(126237,)

In [32]:
rapid_users = pd.concat((pd.Series(games_df[games_df.TimeControl=="Rapid"].BlackName.unique()),
                          pd.Series(games_df[games_df.TimeControl=="Rapid"].WhiteName.unique())))

In [38]:
rapid_users = rapid_users.drop_duplicates()

In [6]:
all_users = pd.concat((pd.Series(games_df.BlackName.unique()),
                          pd.Series(games_df.WhiteName.unique())))

In [9]:
all_users = all_users.drop_duplicates()
all_users.shape

(241692,)

In [11]:
sample_all = all_users.sample(1000).reset_index(drop=True)
sample_all.shape

(1000,)

In [13]:
mask = pd.Series(len(games_df)*[False])

for i in range(len(sample_all)):
    if i % 100 == 0:
        print(i/len(sample_all)*100, "%")
    mask |= games_df.WhiteName == sample_all[i] 
    mask |= games_df.BlackName == sample_all[i]


0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [58]:
sg_df = games_df[mask]

In [59]:
sg_df.shape

(96643, 16)

In [16]:
sg_df.to_csv("sample_games_all.csv")

In [18]:
user_df = pd.DataFrame({"Name":sample_all, 
                        "Num games":sample_all.apply(
                            lambda x: ((sg_df.WhiteName == x) | (sg_df.BlackName == x)).sum())
                       })

In [69]:
umd = {x: ((sg_df.WhiteName == x) | (sg_df.BlackName == x)) for x in sample_all}

In [75]:
sg_df["Date"] = pd.to_datetime(sg_df[["Day","Month","Year"]])

<ipython-input-75-4422f7ee2d55>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sg_df["Date"] = pd.to_datetime(sg_df[["Day","Month","Year"]])


In [25]:
user_df["Start Date"] = sample_all.apply(lambda x : sg_df[user_mask_dict[x]].Date.min())

In [27]:
user_df["End Date"] = sample_all.apply(lambda x : sg_df[user_mask_dict[x]].Date.max())

In [29]:
user_df["Num sessions"] = sample_all.apply(lambda x: sg_df[user_mask_dict[x]].Date.nunique())

107

In [37]:
bmd = {x:(sg_df.BlackName == x) for x in sample_all}
wmd = {x:(sg_df.WhiteName == x) for x in sample_all}

In [38]:
user_df["Black wins"] = sample_all.apply(lambda x : (bmd[x] & (sg_df.Winner == "Black")).sum())

In [39]:
user_df["Black losses"] = sample_all.apply(lambda x : (bmd[x] & (sg_df.Winner == "White")).sum())

In [40]:
user_df["Black draws"] = sample_all.apply(lambda x : (bmd[x] & (sg_df.Winner == "Draw")).sum())

In [41]:
user_df["White wins"] = sample_all.apply(lambda x : (wmd[x] & (sg_df.Winner == "Black")).sum())
user_df["White losses"] = sample_all.apply(lambda x : (wmd[x] & (sg_df.Winner == "White")).sum())
user_df["White draws"] = sample_all.apply(lambda x : (wmd[x] & (sg_df.Winner == "Draw")).sum())

In [46]:
user_df.sample(10)

,Name,Num games,Start Date,End Date,Num sessions,Black wins,Black losses,Black draws,White wins,White losses,White draws
688,JustGame,2,2014-12-20,2014-12-20,1,1,0,0,0,1,0
373,India_Madein,5,2014-07-23,2014-07-23,1,1,1,0,2,1,0
408,IVANFFFFFF,2,2014-07-01,2014-07-01,1,0,1,0,1,0,0
616,ir1,1,2013-06-12,2013-06-12,1,0,1,0,0,0,0
919,ZssZii,3,2014-08-12,2014-08-12,1,2,0,0,0,1,0
979,Klavierkabarett,1,2013-08-29,2013-08-29,1,0,1,0,0,0,0
473,kerbook,1,2014-02-04,2014-02-04,1,0,1,0,0,0,0
602,Afterimage,710,2013-09-07,2014-10-06,81,210,192,10,144,144,10
549,IvanDmC,2,2014-06-07,2014-06-07,1,0,1,0,1,0,0
116,proust1810,17,2013-12-18,2013-12-20,3,0,7,1,6,3,0


In [47]:
user_df.to_csv("user_data.csv")

In [44]:
in_both = sg_df.WhiteName.apply(lambda x: x in sample_all ) & sg_df.BlackName.apply(lambda x: x in sample_all)



0

In [148]:
#Do this in excel

#user_df["Wins"] = user_df["Black wins"] + user_df["White wins"]
#user_df["Losses"] = user_df["Black losses"] + user_df["White losses"]
#user_df["Draws"] = user_df["Black draws"] + user_df["White draws"]

In [157]:
#sg_df["EloDiff"] = sg_df["WhiteElo"] - sg_df["BlackElo"]

<ipython-input-157-86799e55265e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sg_df["EloDiff"] = sg_df["WhiteElo"] - sg_df["BlackElo"]


In [ ]:
#user_df.to_csv("sample_user_data_rapid.csv")

In [159]:
#umd = user_mask_dict

In [162]:
#test_user = "radaam"

In [182]:
def first_game_info(x):
    game1 = sg_df[umd[x]].sort_values("Date").head(1).reset_index(drop=True)

    if game1.WhiteName[0] == x:
        color1 = "White"
    else:
        color1 = "Black"
    
    if game1.Winner[0] == "Draw":
        result1 = "Draw"
    elif game1.Winner[0] == color1:
        result1 = "Win"
    else:
        result1 = "Loss"
    
    if color1 == "White":
        elodiff1 = game1.EloDiff[0]
    else:
        elodiff1 = -1*game1.EloDiff[0]
    
    return (x, color1, result1, elodiff1)

columns = ["Name", "First Color", "First Result", "First Elo Diff"]
game1_series = user_df.Name.apply(first_game_info)

first_df = pd.DataFrame([[t[0],t[1],t[2],t[3]] for t in game1_series.values], columns = columns)

In [183]:
first_df.head()

,Name,First Color,First Result,First Elo Diff
0,tseref,Black,Loss,-238
1,kanon2,Black,Win,-221
2,Velocity99,Black,Loss,-113
3,RajaNarendra,Black,Loss,-53
4,ji419,Black,Loss,-62


In [185]:
df = user_df.join(first_df.set_index("Name"), on="Name")

In [186]:
df.head()

,Name,Num games,Start Date,End Date,Days between,Black wins,Black losses,Black draws,White wins,White losses,White draws,Wins,Losses,Draws,First Color,First Result,First Elo Diff
0,tseref,1,2013-09-23,2013-09-23,0 days,0,1,0,0,0,0,0,1,0,Black,Loss,-238
1,kanon2,31,2014-07-13,2014-07-13,0 days,5,13,0,6,7,0,11,20,0,Black,Win,-221
2,Velocity99,1,2014-09-07,2014-09-07,0 days,0,1,0,0,0,0,0,1,0,Black,Loss,-113
3,RajaNarendra,3,2014-03-25,2014-04-10,16 days,0,2,0,1,0,0,1,2,0,Black,Loss,-53
4,ji419,1,2014-06-11,2014-06-11,0 days,0,1,0,0,0,0,0,1,0,Black,Loss,-62


In [187]:
df.to_csv("rapid_users.csv")

Modify the games data frame so that it is centered on our user from our sample. In terms of them.

In [54]:
#Player name, Opponent, Result, Player ELO 
#Opponent ELO, first game?, final game?, first session?, final session?

#def get_name(row):
#    if row["WhiteName"] = sample_all:
#        return row["WhiteName"]
#    else:
#        return row["BlackName"]

#sg_df["Player name"] = sg_df.apply(get_name, axis=1)

<ipython-input-54-63514c2f07e2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sg_df["Player name"] = sg_df.apply(get_name, axis=1)


In [72]:
sg_df.head()

,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
938,1756,1741,traker100,ptdhina,White,Time forfeit,https://lichess.org/4zu4576l,1,1,2013,240,0,Blitz,Queen's Gambit Declined,D30,26
943,1766,1555,traker100,akayhan,White,Time forfeit,https://lichess.org/oqqk548g,1,1,2013,240,0,Blitz,Queen's Gambit Accepted: Saduleto Variation,D20,35
963,1772,1630,traker100,flatron1,White,Normal,https://lichess.org/2nx8xdle,1,1,2013,240,0,Blitz,Queen's Gambit Declined,D30,58
992,1778,1672,traker100,sempes,White,Normal,https://lichess.org/9d0ytl2q,1,1,2013,240,0,Blitz,Queen's Gambit Accepted: Saduleto Variation,D20,25
1011,1786,1630,traker100,Alex911,Black,Normal,https://lichess.org/nt4otgpt,1,1,2013,240,0,Blitz,Queen's Gambit Refused: Marshall Defense,D06,60


In [117]:
def get_row(old_row):
    new_row = []
    new_row.append(user)
    if old_row.WhiteName == user:
        new_row.append(old_row.BlackName)
        new_row.append("White")
        new_row.append(old_row.WhiteElo)
        new_row.append(old_row.BlackElo)
    else:
        new_row.append(old_row.WhiteName)
        new_row.append("Black")
        new_row.append(old_row.BlackElo)
        new_row.append(old_row.WhiteElo)
    if old_row.Winner == "Draw":
        new_row.append("Draw")
    elif old_row.Winner == new_row[2]:
        new_row.append("Win")
    else:
        new_row.append("Loss")
    
    return new_row

keeping_columns = ["Termination", "Date", "InitialTime", "Increment", "TimeControl","Opening", "ECO","Number_of_Moves"]
new_columns = ["Player","Opponent","Color", "PlayerElo","OpponentElo","Result"]

ug_df = pd.DataFrame(columns = new_columns + keeping_columns)
for user in sample_all:
    games = sg_df[umd[user]]
    rows = games.apply(get_row, axis=1)
    
    df_temp = pd.merge(pd.DataFrame(list(rows),columns = new_columns),games.reset_index()[keeping_columns],left_index=True,right_index=True)
    ug_df = pd.concat((ug_df,df_temp))
    


In [119]:
ug_df.to_csv("user_games_all.csv")